In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import cv2
import os

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
# faster rcnn model이 포함된 library
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from torch.utils.data import DataLoader, Dataset
import pandas as pd
from tqdm import tqdm

import gc

from ensemble_boxes import *

In [2]:
class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir, transforms=None):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        
        image_id = self.coco.getImgIds(imgIds=index)

        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        boxes = np.array([x['bbox'] for x in anns])

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        labels = np.array([x['category_id'] for x in anns])
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        areas = np.array([x['area'] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)
                                
        is_crowds = np.array([x['iscrowd'] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)
                                
        segmentation = np.array([x['segmentation'] for x in anns], dtype=object)

        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([index]), 'area': areas,
                  'iscrowd': is_crowds}

        # transform
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = torch.tensor(sample['bboxes'], dtype=torch.float32)

        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [3]:
def get_train_transform():
    return A.Compose([
        A.Resize(512, 512),
        A.Flip(p=0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform():
    return A.Compose([
        A.Resize(512, 512),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


In [4]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


def collate_fn(batch):
    return tuple(zip(*batch))


In [5]:
def make_ensemble_predictions(images):
    checkpoints = ['./work_dirs/detectors_r50_1x_trash/fold0/epoch_13.pth', './work_dirs/detectors_r50_1x_trash/fold1/epoch_13.pth', './work_dirs/detectors_r50_1x_trash/fold2/epoch_13.pth', './work_dirs/detectors_r50_1x_trash/fold3/epoch_13.pth', './work_dirs/detectors_r50_1x_trash/fold4/epoch_13.pth']
    result = []
    for checkpoint in checkpoints:
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        images = list(image.float().to(device) for image in images) 
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        
        num_classes = 11  # 11 class + background

        # get number of input features for the classifier
        in_features = model.roi_heads.box_predictor.cls_score.in_features

        # torchvision에 학습되어 있던 RPN 사용
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        model.to(device)
        
        model.load_state_dict(torch.load(checkpoint))
        model.eval()

        outputs = model(images)
        result.append(outputs)

        del checkpoint, model, device
        gc.collect()
    
    return result

In [6]:
def run_wbf(predictions, image_index, image_size=512, iou_thr=0.55, skip_box_thr=0.5, weights=None):
    box = [prediction[image_index]['boxes'].data.cpu().numpy()/(image_size-1) for prediction in predictions]
    score = [prediction[image_index]['scores'].data.cpu().numpy() for prediction in predictions]
    label = [prediction[image_index]['labels'].data.cpu().numpy() for prediction in predictions]

    box, score, label = weighted_boxes_fusion(box, score, label, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    box = box*(image_size-1)

    return box, score, label

In [7]:
score_threshold = 0.05
annotation = '../../input/data/test.json'
def valid_fn(val_data_loader):
    outputs = []
    prediction_strings = []
    file_names = []
    coco = COCO(annotation)

    for images, targets, image_ids in tqdm(val_data_loader):
        # gpu 계산을 위해 image.to(device)
        predictions = make_ensemble_predictions(images)
        for i, (image_id, image) in enumerate(zip(image_ids, images)):
            prediction_string = ''
            image_info = coco.loadImgs(coco.getImgIds(imgIds=image_id[0]))[0]
            boxes, scores, labels = run_wbf(predictions, image_index=i)
            for box, score, label in zip(boxes, scores, labels):
                if score > score_threshold:
                        prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
                            box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
            if prediction_string != '':
                prediction_strings.append(prediction_string)
                file_names.append(image_info['file_name'])
    
        del predictions
        gc.collect()
    

    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = file_names
    submission.to_csv(f'k-fold_detectors.csv', index=None)
    print(submission.head())

In [8]:
def main():
    data_dir = '../../input/data'
    val_dataset = CustomDataset(annotation, data_dir, get_valid_transform())
    val_data_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=4,
        collate_fn=collate_fn
    )
    valid_fn(val_data_loader)

In [9]:
if __name__ == '__main__':
    main()

  0%|          | 0/105 [00:09<?, ?it/s]


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


RuntimeError: Error(s) in loading state_dict for FasterRCNN:
	Missing key(s) in state_dict: "backbone.body.conv1.weight", "backbone.body.bn1.weight", "backbone.body.bn1.bias", "backbone.body.bn1.running_mean", "backbone.body.bn1.running_var", "backbone.body.layer1.0.conv1.weight", "backbone.body.layer1.0.bn1.weight", "backbone.body.layer1.0.bn1.bias", "backbone.body.layer1.0.bn1.running_mean", "backbone.body.layer1.0.bn1.running_var", "backbone.body.layer1.0.conv2.weight", "backbone.body.layer1.0.bn2.weight", "backbone.body.layer1.0.bn2.bias", "backbone.body.layer1.0.bn2.running_mean", "backbone.body.layer1.0.bn2.running_var", "backbone.body.layer1.0.conv3.weight", "backbone.body.layer1.0.bn3.weight", "backbone.body.layer1.0.bn3.bias", "backbone.body.layer1.0.bn3.running_mean", "backbone.body.layer1.0.bn3.running_var", "backbone.body.layer1.0.downsample.0.weight", "backbone.body.layer1.0.downsample.1.weight", "backbone.body.layer1.0.downsample.1.bias", "backbone.body.layer1.0.downsampl